# 🧭 TSP: Solução Clássica vs QAOA
## Comparação entre Brute Force e Algoritmo Quântico

Este notebook compara a solução do Problema do Caixeiro Viajante (TSP) usando:
1. **Brute Force Clássico** - Encontra a solução ótima testando todas as permutações
2. **QAOA (Quantum Approximate Optimization Algorithm)** - Algoritmo quântico aproximado

## 📦 Célula 1: Instalação das Dependências
Execute esta célula apenas uma vez para instalar os pacotes necessários.

In [ ]:
# Instalação dos pacotes necessários
# Descomente e execute se ainda não tiver instalado

!pip install qiskit==1.2.4
!pip install qiskit-optimization==0.6.1
!pip install qiskit-algorithms==0.3.0
!pip install qiskit-aer==0.15.1
!pip install numpy matplotlib

## 📚 Célula 2: Importações

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import permutations
import time
import warnings
warnings.filterwarnings('ignore')

# Verificar se Qiskit está disponível
QISKIT_AVAILABLE = False
try:
    from qiskit_optimization.applications import Tsp
    from qiskit_optimization.converters import QuadraticProgramToQubo
    from qiskit_algorithms import QAOA
    from qiskit_algorithms.optimizers import COBYLA, SPSA
    from qiskit.primitives import Sampler
    from qiskit_optimization.algorithms import MinimumEigenOptimizer
    QISKIT_AVAILABLE = True
    print("✅ Qiskit instalado e funcionando!")
except ImportError as e:
    print(f"⚠️ Qiskit não disponível: {e}")
    print("Execute a célula de instalação acima e reinicie o kernel.")

## 🔧 Célula 3: Funções Auxiliares

In [ ]:
def criar_matriz_distancias(n_cidades, seed=42):
    """
    Cria uma matriz de distâncias simétrica para n cidades.
    """
    np.random.seed(seed)
    matriz = np.random.randint(5, 30, size=(n_cidades, n_cidades))
    matriz = (matriz + matriz.T) // 2  # Torna simétrica
    np.fill_diagonal(matriz, 0)  # Diagonal = 0
    return matriz.astype(float)


def calcular_custo_rota(rota, matriz):
    """
    Calcula o custo total de uma rota.
    """
    custo = 0
    for i in range(len(rota) - 1):
        custo += matriz[rota[i]][rota[i + 1]]
    return custo


def brute_force_tsp(matriz):
    """
    Resolve TSP por força bruta (testando todas as permutações).
    """
    n = len(matriz)
    cidades = list(range(n))
    
    melhor_rota = None
    melhor_custo = float('inf')
    
    # Fixa a cidade 0 como início para evitar rotas equivalentes
    for perm in permutations(cidades[1:]):
        rota = (0,) + perm + (0,)  # Começa e termina em 0
        custo = calcular_custo_rota(rota, matriz)
        
        if custo < melhor_custo:
            melhor_custo = custo
            melhor_rota = rota
    
    return melhor_rota, melhor_custo


def visualizar_rota(matriz, rota, titulo="Rota TSP"):
    """
    Visualiza a rota em um grafo.
    """
    n = len(matriz)
    
    # Gera posições das cidades em círculo
    angulos = np.linspace(0, 2 * np.pi, n, endpoint=False)
    posicoes = {i: (np.cos(a), np.sin(a)) for i, a in enumerate(angulos)}
    
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    
    # Desenha todas as arestas (cinza claro)
    for i in range(n):
        for j in range(i + 1, n):
            x = [posicoes[i][0], posicoes[j][0]]
            y = [posicoes[i][1], posicoes[j][1]]
            ax.plot(x, y, 'lightgray', linewidth=0.5, zorder=1)
    
    # Desenha a rota (azul)
    for i in range(len(rota) - 1):
        x = [posicoes[rota[i]][0], posicoes[rota[i + 1]][0]]
        y = [posicoes[rota[i]][1], posicoes[rota[i + 1]][1]]
        ax.plot(x, y, 'b-', linewidth=2, zorder=2)
        
        # Adiciona seta
        mid_x = (x[0] + x[1]) / 2
        mid_y = (y[0] + y[1]) / 2
        dx = x[1] - x[0]
        dy = y[1] - y[0]
        ax.annotate('', xy=(mid_x + dx * 0.1, mid_y + dy * 0.1),
                    xytext=(mid_x - dx * 0.1, mid_y - dy * 0.1),
                    arrowprops=dict(arrowstyle='->', color='blue', lw=1.5))
    
    # Desenha os nós
    for i, (x, y) in posicoes.items():
        cor = 'red' if i == 0 else 'green'
        ax.scatter(x, y, s=500, c=cor, zorder=3, edgecolors='black')
        ax.text(x, y, str(i), fontsize=12, ha='center', va='center', 
                fontweight='bold', color='white', zorder=4)
    
    ax.set_title(titulo, fontsize=14, fontweight='bold')
    ax.set_aspect('equal')
    ax.axis('off')
    
    return fig

## 🔬 Célula 4: Função QAOA para TSP

In [ ]:
def qaoa_tsp(matriz, reps=2):
    """
    Resolve TSP usando QAOA do Qiskit.
    
    Args:
        matriz: Matriz de distâncias
        reps: Número de repetições do circuito QAOA
    
    Returns:
        tuple: (rota, custo) ou (None, None) se falhar
    """
    if not QISKIT_AVAILABLE:
        return None, None, "Qiskit não disponível"
    
    try:
        n = len(matriz)
        
        # Cria o problema TSP
        tsp = Tsp(matriz)
        qp = tsp.to_quadratic_program()
        
        # Configura o QAOA
        sampler = Sampler()
        optimizer = COBYLA(maxiter=100)
        
        qaoa = QAOA(
            sampler=sampler,
            optimizer=optimizer,
            reps=reps
        )
        
        # Resolve
        algorithm = MinimumEigenOptimizer(qaoa)
        result = algorithm.solve(qp)
        
        # Interpreta o resultado
        x = result.x
        
        # Reconstrói a rota a partir da solução
        rota = tsp.interpret(result)
        
        # Calcula o custo real
        rota_completa = tuple(rota) + (rota[0],)
        custo = calcular_custo_rota(rota_completa, matriz)
        
        return rota_completa, custo, None
        
    except Exception as e:
        return None, None, str(e)

## 🚀 Célula 5: Execução Principal

In [ ]:
def executar_comparacao(n_cidades=4, seed=42):
    """
    Executa a comparação completa entre Brute Force e QAOA.
    """
    print("=" * 80)
    print("TSP: SOLUÇÃO CLÁSSICA vs QAOA")
    print("=" * 80)
    print(f"\n📍 Número de cidades: {n_cidades}")
    print(f"🎲 Seed aleatória: {seed}\n")
    
    # Cria a matriz de distâncias
    matriz = criar_matriz_distancias(n_cidades, seed)
    
    print("📊 Matriz de Distâncias:")
    print(matriz.astype(int))
    print()
    
    resultados = {}
    
    # === BRUTE FORCE ===
    print("-" * 40)
    print("[1/2] 🔍 Executando Brute Force Clássico...")
    print("-" * 40)
    
    inicio = time.time()
    rota_bf, custo_bf = brute_force_tsp(matriz)
    tempo_bf = time.time() - inicio
    
    print(f"     ✅ Rota ótima: {rota_bf}")
    print(f"     ✅ Custo: {custo_bf:.4f}")
    print(f"     ✅ Tempo: {tempo_bf:.6f}s")
    
    resultados['brute_force'] = {
        'rota': rota_bf,
        'custo': custo_bf,
        'tempo': tempo_bf
    }
    
    # === QAOA ===
    print()
    print("-" * 40)
    print("[2/2] ⚛️  Executando QAOA...")
    print("-" * 40)
    
    if QISKIT_AVAILABLE:
        inicio = time.time()
        rota_qaoa, custo_qaoa, erro = qaoa_tsp(matriz, reps=2)
        tempo_qaoa = time.time() - inicio
        
        if erro:
            print(f"     ❌ Erro: {erro}")
            resultados['qaoa'] = {'erro': erro}
        else:
            print(f"     ✅ Rota encontrada: {rota_qaoa}")
            print(f"     ✅ Custo: {custo_qaoa:.4f}")
            print(f"     ✅ Tempo: {tempo_qaoa:.6f}s")
            
            resultados['qaoa'] = {
                'rota': rota_qaoa,
                'custo': custo_qaoa,
                'tempo': tempo_qaoa
            }
    else:
        print("     ⚠️  Qiskit não está disponível.")
        print("     💡 Execute a célula de instalação e reinicie o kernel.")
        resultados['qaoa'] = {'erro': 'Qiskit não disponível'}
    
    # === RESUMO ===
    print()
    print("=" * 80)
    print("📊 RESUMO COMPARATIVO")
    print("=" * 80)
    
    print(f"\n{'Método':<20} {'Rota':<25} {'Custo':<12} {'Tempo (s)':<12}")
    print("-" * 70)
    
    print(f"{'Brute Force':<20} {str(rota_bf):<25} {custo_bf:<12.4f} {tempo_bf:<12.6f}")
    
    if 'erro' not in resultados.get('qaoa', {}):
        r = resultados['qaoa']
        print(f"{'QAOA':<20} {str(r['rota']):<25} {r['custo']:<12.4f} {r['tempo']:<12.6f}")
        
        # Análise
        print()
        if r['custo'] == custo_bf:
            print("🎯 QAOA encontrou a solução ÓTIMA!")
        else:
            gap = ((r['custo'] - custo_bf) / custo_bf) * 100
            print(f"📈 Gap de otimalidade: {gap:.2f}%")
    else:
        print(f"{'QAOA':<20} {'N/A':<25} {'N/A':<12} {'N/A':<12}")
    
    return matriz, resultados


# EXECUTAR!
matriz, resultados = executar_comparacao(n_cidades=4, seed=42)

## 📈 Célula 6: Visualização

In [ ]:
# Visualiza as rotas encontradas
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Rota Brute Force
rota_bf = resultados['brute_force']['rota']
custo_bf = resultados['brute_force']['custo']

n = len(matriz)
angulos = np.linspace(0, 2 * np.pi, n, endpoint=False)
posicoes = {i: (np.cos(a), np.sin(a)) for i, a in enumerate(angulos)}

ax = axes[0]
for i in range(n):
    for j in range(i + 1, n):
        x = [posicoes[i][0], posicoes[j][0]]
        y = [posicoes[i][1], posicoes[j][1]]
        ax.plot(x, y, 'lightgray', linewidth=0.5)

for i in range(len(rota_bf) - 1):
    x = [posicoes[rota_bf[i]][0], posicoes[rota_bf[i + 1]][0]]
    y = [posicoes[rota_bf[i]][1], posicoes[rota_bf[i + 1]][1]]
    ax.plot(x, y, 'b-', linewidth=2.5)

for i, (x, y) in posicoes.items():
    cor = 'red' if i == 0 else 'green'
    ax.scatter(x, y, s=500, c=cor, edgecolors='black', zorder=5)
    ax.text(x, y, str(i), fontsize=12, ha='center', va='center', 
            fontweight='bold', color='white', zorder=6)

ax.set_title(f'Brute Force (Ótimo)\nRota: {rota_bf}\nCusto: {custo_bf}', 
             fontsize=12, fontweight='bold')
ax.set_aspect('equal')
ax.axis('off')

# Rota QAOA
ax = axes[1]
if 'erro' not in resultados.get('qaoa', {}):
    rota_qaoa = resultados['qaoa']['rota']
    custo_qaoa = resultados['qaoa']['custo']
    
    for i in range(n):
        for j in range(i + 1, n):
            x = [posicoes[i][0], posicoes[j][0]]
            y = [posicoes[i][1], posicoes[j][1]]
            ax.plot(x, y, 'lightgray', linewidth=0.5)
    
    for i in range(len(rota_qaoa) - 1):
        x = [posicoes[rota_qaoa[i]][0], posicoes[rota_qaoa[i + 1]][0]]
        y = [posicoes[rota_qaoa[i]][1], posicoes[rota_qaoa[i + 1]][1]]
        ax.plot(x, y, 'purple', linewidth=2.5)
    
    for i, (x, y) in posicoes.items():
        cor = 'red' if i == 0 else 'green'
        ax.scatter(x, y, s=500, c=cor, edgecolors='black', zorder=5)
        ax.text(x, y, str(i), fontsize=12, ha='center', va='center', 
                fontweight='bold', color='white', zorder=6)
    
    status = "✅ Ótimo!" if custo_qaoa == custo_bf else f"📈 Gap: {((custo_qaoa-custo_bf)/custo_bf)*100:.1f}%"
    ax.set_title(f'QAOA\nRota: {rota_qaoa}\nCusto: {custo_qaoa} {status}', 
                 fontsize=12, fontweight='bold')
else:
    ax.text(0.5, 0.5, 'QAOA não disponível\n\nInstale o Qiskit e\nreinicie o kernel', 
            ha='center', va='center', fontsize=14, transform=ax.transAxes)
    ax.set_title('QAOA', fontsize=12, fontweight='bold')

ax.set_aspect('equal')
ax.axis('off')

plt.tight_layout()
plt.show()

## 🧪 Célula 7: Teste com Diferentes Tamanhos

In [ ]:
# Teste com diferentes números de cidades
# ATENÇÃO: QAOA fica muito lento para n > 5 cidades!

print("Testando diferentes tamanhos de problema...\n")
print(f"{'Cidades':<10} {'Custo BF':<12} {'Tempo BF':<12} {'Custo QAOA':<12} {'Tempo QAOA':<12} {'Match?':<10}")
print("-" * 70)

for n in [3, 4, 5]:
    matriz = criar_matriz_distancias(n, seed=42)
    
    # Brute Force
    inicio = time.time()
    rota_bf, custo_bf = brute_force_tsp(matriz)
    tempo_bf = time.time() - inicio
    
    # QAOA
    if QISKIT_AVAILABLE:
        inicio = time.time()
        rota_qaoa, custo_qaoa, erro = qaoa_tsp(matriz, reps=2)
        tempo_qaoa = time.time() - inicio
        
        if erro:
            custo_qaoa = "Erro"
            tempo_qaoa = "N/A"
            match = "N/A"
        else:
            match = "✅" if custo_qaoa == custo_bf else "❌"
            custo_qaoa = f"{custo_qaoa:.2f}"
            tempo_qaoa = f"{tempo_qaoa:.4f}"
    else:
        custo_qaoa = "N/A"
        tempo_qaoa = "N/A"
        match = "N/A"
    
    print(f"{n:<10} {custo_bf:<12.2f} {tempo_bf:<12.6f} {str(custo_qaoa):<12} {str(tempo_qaoa):<12} {match:<10}")

## 📝 Notas

### Limitações do QAOA:
- **Problema NP-hard**: O TSP é exponencialmente difícil
- **Escalabilidade**: QAOA requer n² qubits para n cidades
- **Aproximação**: QAOA nem sempre encontra a solução ótima
- **Simulação**: Estamos simulando em computador clássico

### Recomendações:
- Use no máximo 5 cidades para testes rápidos
- Para problemas maiores, considere heurísticas clássicas
- O verdadeiro potencial quântico aparece em hardware real